In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [0]:
with open('/content/drive/My Drive/NLP/train.txt','r') as f:
    data_train = f.readlines()
    f.close()
with open('/content/drive/My Drive/NLP/test.txt','r') as f:
    data_test = f.readlines()
    f.close()

In [0]:
test_data_pd = pd.DataFrame(columns=['uid','sentiment','sentence','lang_id'])
train_data_pd = pd.DataFrame(columns=['uid','sentiment','sentence','lang_id'])
sent_code={'neutral':0,'positive':1,'negative':2}

In [0]:
for i in range(len(data_train)):
    temp = data_train[i].split()
    if len(temp) in [0,1]:
        continue
    if temp[0]=='meta':
        if i:
            train_data_pd=train_data_pd.append({'uid':uid,'sentiment':senti,'sentence':' '.join(s),'lang_id':' '.join(lang)},ignore_index=True)
        uid = int(temp[1])
        senti = sent_code[temp[2]]
        s = []
        lang = []
        continue
    if temp[1] in ['Eng','Hin']:
        s.append(temp[0])
        lang.append(temp[1])

In [0]:
train_data_pd.to_csv(r'/content/drive/My Drive/NLP/train.csv',index=None,header=True)

In [0]:
for i in range(len(data_test)):
    temp = data_test[i].split()
    if len(temp) in [0,1]:
        continue
    if temp[0]=='meta':
        if i:
            test_data_pd=test_data_pd.append({'uid':uid,'sentiment':senti,'sentence':' '.join(s),'lang_id':' '.join(lang)},ignore_index=True)
        uid = int(temp[1])
        senti = sent_code[temp[2]]
        s = []
        lang = []
        continue
    if temp[1] in ['Eng','Hin']:
        s.append(temp[0])
        lang.append(temp[1])

In [0]:
test_data_pd.to_csv(r'/content/drive/My Drive/NLP/test.csv',index=None,header=True)

In [0]:
train_data_pd = pd.read_csv('/content/drive/My Drive/NLP/train.csv')
test_data_pd = pd.read_csv('/content/drive/My Drive/NLP/test.csv')

In [0]:
train_X,train_Y = list(train_data_pd['sentence']),list(train_data_pd['sentiment'])

In [0]:
test_X,test_Y = list(test_data_pd['sentence']),list(test_data_pd['sentiment'])

In [0]:
vocab={}
ind = 1
for s in train_X:
  sw = s.split()
  for w in sw:
    w.lower()
    if w not in vocab.keys():
      vocab[w]=ind
      ind+=1

In [0]:
# train_encod = np.zeros((len(train_X),1,256))
# test_encod = np.zeros((len(test_X),1,256))

In [0]:
# for i in range(len(train_X)):
#   s = train_X[i].lower()
#   sw = s.split()
#   for j in range(len(sw)):
#     if sw[j] in vocab.keys():
#       train_encod[i,0,j]=vocab[sw[j]]
# for i in range(len(test_X)):
#   s = test_X[i].lower()
#   sw = s.split()
#   for j in range(len(sw)):
#     if sw[j] in vocab.keys():
#       test_encod[i,0,j]=vocab[sw[j]]

In [0]:
train_encod2 = np.zeros((len(train_X),1,len(vocab)+1))
test_encod2 = np.zeros((len(test_X),1,len(vocab)+1))

In [0]:
for i in range(len(train_X)):
  s = train_X[i].lower()
  sw = s.split()
  for w in sw:
    if w in vocab.keys():
      train_encod2[i,0,vocab[w]]+=1
for i in range(len(test_X)):
  s = test_X[i].lower()
  sw = s.split()
  for w in sw:
    if w in vocab.keys():
      test_encod2[i,0,vocab[w]]+=1

In [6]:
!pip install keras-bert
from keras_bert import extract_embeddings

  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=6aa2ede8c418d9b941ac8b612e205690f3cc7fe9edd221231e6ef327c40dfb7d
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=5426b68f672834370a05e844f84caca11030346b13e89ddd26074273ffb07b83
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=c46627eac6328a05b380cbe733f0f017a35fabbdff2ca4dfe8435d9ecc61e13a
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=05e6462764206c26d349c2ef6eb509bf636002335683e23f8f2ab4cd0eb23afa
  Stor

Using TensorFlow backend.


In [0]:
bert_path = '/content/drive/My Drive/NLP/multi_cased_L-12_H-768_A-12'

In [8]:
temp_emb1 = extract_embeddings(bert_path,train_X)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









KeyboardInterrupt: ignored

In [0]:
train_emb = np.zeros((len(temp_emb1),1,768))
for i in range(len(temp_emb)):
  train_emb[i,0,:] = np.mean(temp_emb1[i][1:-1],axis=0)

In [0]:
temp_emb2 = extract_embeddings(bert_path,test_X)

In [0]:
test_emb = np.zeros((len(temp_emb2),1,768))
for i in range(len(temp_emb2)):
  test_emb[i,0,:] = np.mean(temp_emb2[i][1:-1],axis=0)

In [19]:
from sklearn.preprocessing import OneHotEncoder
encod = OneHotEncoder(sparse=False)
train_labels = encod.fit_transform(np.array(train_Y).reshape(-1,1))
test_labels = encod.fit_transform(np.array(test_Y).reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [38]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
# model = Sequential([
#                     LSTM(1024,input_shape=(768,),activation='relu',return_sequences=True),
#                     LSTM(512,activation='relu',return_sequences=True),
#                     Dropout(0.3),
#                     Dense(128),
#                     Activation('relu'),
#                     Dense(32),
#                     Activation('relu'),
#                     Dense(3),
#                     Activation('softmax')
# ])
model=Sequential()
# model.add(LSTM(512,input_shape=(1,256),activation='relu',return_sequences=True))
model.add(Dense(1024,input_shape=(1,len(vocab)+1),activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
# model.add(Dense(32,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [39]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 1, 1024)           60305408  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1024)           0         
_________________________________________________________________
dense_14 (Dense)             (None, 1, 128)            131200    
_________________________________________________________________
dense_15 (Dense)             (None, 1, 3)              387       
Total params: 60,436,995
Trainable params: 60,436,995
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(train_encod2, np.expand_dims(train_labels,axis=1), batch_size=32,  epochs =10)

Epoch 1/10
15130/15130 [==============================] - 19s 1ms/step - loss: 0.8636 - acc: 0.5870
Epoch 2/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.4810 - acc: 0.8106
Epoch 3/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.1500 - acc: 0.9505
Epoch 4/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.0578 - acc: 0.9792
Epoch 5/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.0325 - acc: 0.9839
Epoch 6/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.0265 - acc: 0.9849
Epoch 7/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.0208 - acc: 0.9868
Epoch 8/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.0193 - acc: 0.9880
Epoch 9/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.0209 - acc: 0.9872
Epoch 10/10
15130/15130 [==============================] - 17s 1ms/step - loss: 0.0214 - acc: 0.9876

In [0]:
pred_label = model.predict(test_encod2)

In [0]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

In [68]:
accuracy_score(np.argmax(test_labels,axis=1),np.argmax(pred_label,axis=2))

0.5240899357601713

In [66]:
precision_recall_fscore_support(np.argmax(test_labels,axis=1),np.argmax(pred_label,axis=2),average='macro')

(0.5289722790792104, 0.5239681372298731, 0.5254338948102534, None)

Tried finetuning BERT by using pretrained Multilingual BERT model. Code taken enitrely from https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb. This was done only to check how well this model performs compared to mine. But unfortunately the model is done for binary classification and multi-class classification is not available.

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras

In [71]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.3MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
train_InputExamples = train_data_pd.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['sentence'], 
                                                                   text_b = None, 
                                                                   label = x['sentiment']), axis = 1)

In [74]:
train_data_pd['sentiment'].unique()

array([2, 1, 0])

In [0]:
test_InputExamples = test_data_pd.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['sentence'], 
                                                                   text_b = None, 
                                                                   label = x['sentiment']), axis = 1)

In [76]:
tokenizer = bert.tokenization.FullTokenizer(
      vocab_file='/content/drive/My Drive/NLP/multi_cased_L-12_H-768_A-12/vocab.txt', do_lower_case=True)

In [77]:
MAX_SEQ_LENGTH = 128
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, [0,1,2], MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, [0,1,2], MAX_SEQ_LENGTH, tokenizer)


INFO:tensorflow:Writing example 0 of 15130
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ad ##il ##nis ##ar ##but ##t pak ##istan ka gh ##ra tau ##q he pak ##istan is ##rae ##l ko tas ##lee ##m nahe ##in ker ##ta is ##ko pale ##stin ##e ke ##hta he occupied pale ##stin ##e [SEP]
INFO:tensorflow:input_ids: 101 10840 11030 12597 10354 23170 10123 17284 35631 10730 25327 10288 73321 11703 10261 17284 35631 10124 73803 10161 11252 30853 30188 10147 30037 10245 53628 10213 10124 10440 30610 49529 10112 11163 32493 10261 28634 30610 49529 10112 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1",
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
run_config = tf.estimator.RunConfig(
    model_dir='/content/drive/My Drive/NLP/',
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [87]:
model_fn = model_fn_builder(
  num_labels=3,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/NLP/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f24a60b26a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore



/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



...

INFO:tensorflow:loss = 0.76663715, step = 200 (48.663 sec)


INFO:tensorflow:global_step/sec: 2.05547


INFO:tensorflow:global_step/sec: 2.05547


INFO:tensorflow:loss = 0.89335036, step = 300 (48.651 sec)


INFO:tensorflow:loss = 0.89335036, step = 300 (48.651 sec)


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
pred = estimator.evaluate(input_fn=test_input_fn, steps=None)